In [1]:
import pandas as pd
import glob
from tqdm import tqdm

In [2]:
import json
import dask.bag as db

In [5]:
df = pd.read_json('transaction.9013', lines=True, chunksize=10000)
df_ = pd.concat(df, ignore_index=True)
norm = pd.json_normalize(df_['data'])
norm.to_pickle(f'pickle/transaction.9013.pkl')

In [3]:
pkl_list = []
for x in tqdm(glob.glob('pickle/*')):
    pkl_list.append(pd.read_pickle(x))

100%|██████████| 7/7 [01:26<00:00, 12.41s/it]


In [4]:
filter_list = []
for x in tqdm(pkl_list):
    _from = x._from.split('/')[0] == 'userid'
    _to = x._to.split('/')[0] == 'userid'
    if _from and _to:
        filter_list.append({'from': x._from, 'to': x._to})

  0%|          | 0/7 [00:00<?, ?it/s]


AttributeError: 'Series' object has no attribute 'split'

In [4]:
df_all = pd.concat(pkl_list)

In [ ]:
df_all.to_pickle('pickle/total.pkl')

In [5]:
users = pd.read_csv('./users.csv')
users

,userID,userID_format
0,180224000000004,userid/180224000000004
1,180224000000006,userid/180224000000006
2,200803000254471,userid/200803000254471
3,170405000000077,userid/170405000000077
4,181115000012471,userid/181115000012471
...,...,...
1537072,170811000000096,userid/170811000000096
1537073,200612000038899,userid/200612000038899
1537074,181209000010266,userid/181209000010266
1537075,180821000016033,userid/180821000016033


In [6]:
user_list = users.userID_format.to_list()
user_list[:5]

['userid/180224000000004',
 'userid/180224000000006',
 'userid/200803000254471',
 'userid/170405000000077',
 'userid/181115000012471']

In [7]:
filter_df = df_all.loc[df_all['_from'].isin(user_list) | df_all['_to'].isin(user_list)]

In [8]:
filter_df.reset_index(drop=True, inplace=True)

In [10]:
filter_df.to_json('transaction.json',orient="records")

In [11]:
!jq -c ".[]" transaction.json > transaction.jsonl

In [12]:
!arangoimport --server.username='root' --server.password="" --create-collection true --collection transaction --type jsonl --file "transaction.jsonl" --progress true --threads 32

2020-10-25T13:06:11Z [19505] WARNING capping --threads value to 16
Connected to ArangoDB 'http+tcp://127.0.0.1:8529, version: 3.7.2-1, database: '_system', username: 'root'
----------------------------------------
database:               _system
collection:             transaction
create:                 yes
create database:        no
source filename:        transaction.jsonl
file type:              jsonl
threads:                16
connect timeout:        5
request timeout:        1200
----------------------------------------
Starting JSON import...
2020-10-25T13:06:13Z [19505] INFO [9ddf3] processed 58292493 bytes (3%) of input file
2020-10-25T13:06:16Z [19505] INFO [9ddf3] processed 116584986 bytes (6%) of input file
2020-10-25T13:06:19Z [19505] INFO [9ddf3] processed 174877479 bytes (9%) of input file
2020-10-25T13:06:21Z [19505] INFO [9ddf3] processed 233169972 bytes (12%) of input file
2020-10-25T13:06:23Z [19505] INFO [9ddf3] processed 291462465 bytes (15%) of input file
2020-10-